In [2]:
# Imports

import numpy as np
import numpy.matlib
import pandas as pd
import matplotlib.pyplot as plt

In [295]:
# Get the excel file and read

class ExcelReader(object):

    def __init__(self, path, ecID=1, vals=None):
        self.path = path
        self.ecID = ecID

        self.vals = vals

        # Placeholder dictionary to place the next ones
        self.parameterData = {}
        self.pD_network = {}

        # Resources
        self.resources = {}

        # General Info dictionary
        self.generalInfo = {}

        # Branch dictionary
        self.puSist = {}
        self.network = {}

        self.read_generalInfo()
        self.read_brachData()
        self.read_genData()

        return


    def read_generalInfo(self):
        # General information sheet from excel
        generalInfo = pd.read_excel(self.path, sheet_name='General_Information', header=None)
        self.resources['numEC'] = generalInfo.values[2, 3]
        self.resources['period'] = generalInfo.values[3, 3]
        self.resources['periodDuration'] = generalInfo.values[4, 3]
        self.resources['owners'] = generalInfo.values[5, 3]

        # Read max imports and export quantities
        self.generalInfo['P_Max_Imp'] = generalInfo.values[23, 5:]
        self.generalInfo['P_Max_Exp'] = generalInfo.values[24, 5:]
        self.generalInfo['Energy_Buy_Price'] = generalInfo.values[25, 5:]
        self.generalInfo['Energy_SellPrice'] = generalInfo.values[26, 5:]
        self.generalInfo['GEE_CofA'] = generalInfo.values[27, 5:]
        self.generalInfo['GEE_CofB'] = generalInfo.values[28, 5:]
        self.generalInfo['GEE_CofC'] = generalInfo.values[29, 5:]

        return


    def read_brachData(self):
        # Branch data sheet from excel
        sheet_name_branch = 'Network_Info'
        data_branch = pd.read_excel(self.path, sheet_name=sheet_name_branch, header=None)

        # Set the values
        self.puSist['basePower'] = data_branch.values[1, 8]
        self.puSist['baseVoltage'] = data_branch.values[2, 8]
        self.puSist['baseCurrent'] = data_branch.values[3, 8]
        self.puSist['baseImpendance'] = data_branch.values[4, 8]

        busRef = data_branch.values[2, :3]
        busAux = np.unique(data_branch.values[22:, 1:3])

        voltageMax = np.zeros((max(busAux), self.resources['period']))
        voltageMin = np.zeros((max(busAux), self.resources['period']))
        angleMax = np.zeros((max(busAux), self.resources['period']))
        angleMin = np.zeros((max(busAux), self.resources['period']))

        for i in busAux:
            voltageMax[i-1, :] = data_branch.values[2, 5]
            voltageMin[i-1, :] = data_branch.values[2, 4]
            angleMax[i-1, :] = data_branch.values[4, 5]
            angleMin[i-1, :] = data_branch.values[4, 4]

        voltageMax[busRef[0]-1, :] = busRef[1]
        voltageMin[busRef[0]-1, :] = busRef[1]
        angleMax[busRef[0]-1, :] = busRef[2]
        angleMin[busRef[0]-1, :] = busRef[2]

        self.pD_network['voltageMax'] = voltageMax
        self.pD_network['voltageMin'] = voltageMin
        self.pD_network['angleMax'] = angleMax
        self.pD_network['angleMin'] = angleMin
        self.pD_network['busNum'] = np.reshape(busAux, (1, busAux.shape[0]))
        self.pD_network['busRef'] = busRef[0]

        # Conversion from data_branch
        numBranch = data_branch.values[:, 0].shape[0] - 22
        ybus = np.zeros((max(busAux), max(busAux)))
        diag = np.zeros((max(busAux), max(busAux)))
        branchID = np.zeros((max(busAux), max(busAux)))

        busI = data_branch.values[22:, 1]
        busJ = data_branch.values[22:, 2]

        # Admitance set
        for i in np.arange(busI.shape[0]):
            ybus[busI[i]-1, busJ[i]-1] = 1
            ybus[busJ[i]-1, busI[i]-1] = 1

            branchID[busI[i]-1, busJ[i]-1] = 1


        # Diagonal matrix values
        for i in np.arange(ybus.shape[0]):
            ybus[i, i] = 1
            diag[i, i] = 1

        branch = np.zeros((max(busAux), max(busAux), 4))
        idBusI = np.matlib.repmat(busI, 1, 4)
        idBusI = np.transpose(idBusI)

        idBusJ = np.matlib.repmat(busJ, 1, 4)
        idBusJ = np.transpose(idBusJ)

        # Branch and branchInfo
        mask = np.zeros(branch.shape)
        idBranchInfo = np.matlib.repmat(np.arange(1, 5), numBranch, 1)
        idBranchInfo = np.reshape(idBranchInfo.transpose(), (numBranch*4, 1))
        for i in np.arange(idBranchInfo.shape[0]):
            mask[idBusI[i, 0]-1, idBusJ[i, 0]-1, idBranchInfo[i, 0]-1] = 1

        branch[mask.astype(bool)] = data_branch.values[22:, 5:9].ravel()

        # Line data selection
        lineData = data_branch.values[1:, 0:9]
        lineData = pd.DataFrame(lineData)
        lineData = lineData.loc[(lineData.index != 17) & (lineData.index != 18)]
        for col in lineData.columns:
            lineData[col] = pd.to_numeric(lineData[col], errors='coerce')

        lineData.columns = ['Col{:02d}'.format(i) for i in np.arange(lineData.shape[1])]
        lineData = lineData[['Col00', 'Col01', 'Col02', 'Col05', 'Col06', 'Col07']]

        #Power Limit
        powerLimit = data_branch.values[1:, 0:9]
        powerLimit = pd.DataFrame(powerLimit)
        powerLimit = powerLimit.loc[(powerLimit.index != 17) & (powerLimit.index != 18)]
        for col in powerLimit.columns:
            powerLimit[col] = pd.to_numeric(powerLimit[col], errors='coerce')

        powerLimit.columns = ['Col{:02d}'.format(i) for i in np.arange(powerLimit.shape[1])]
        powerLimit = powerLimit[['Col00', 'Col01', 'Col02', 'Col08']]

        self.network['branch'] = branch
        self.network['lineData'] = lineData.values
        self.network['lineLimit'] = powerLimit
        self.pD_network['ybus'] = ybus
        self.pD_network['diag'] = diag
        self.pD_network['branch'] = branchID
        self.pD_network['numLine'] = numBranch
        return


    def _genOption(self):
        if self.vals == None:
            options = [1, 2, 3, 4, 5, 6, 7,
                       1, 2, 3, 4, 5, 6, 7]
        else:
            options = self.val

        names = ['photovoltaic', 'wind', 'co-generation', 'biomass', 'waste-to-energy', 'small hydro',
                 'fuel cell', 'external supplier', 'none',
                 'Xup1', 'Xup2', 'Active Min', 'Active Max', 'Reactive Min', 'Reactive Max', 'Excess Max']

        idx = {}
        idx['PV'] = options[0]
        idx['Wind'] = options[1]
        idx['CHP'] = options[2]
        idx['Bio'] = options[3]
        idx['WtE'] = options[4]
        idx['SH'] = options[5]
        idx['FC'] = options[6]
        idx['ES'] = options[7]
        idx['No'] = options[8]

        return options, names, idx


    def read_genData(self):
        # Read the Excel
        sheet_name_gen = 'Generator_EC{}'.format(self.ecID)
        data_gen = pd.read_excel(self.path, sheet_name=sheet_name_gen, header=None)

        # Conversion from genData
        genOpt, genOptName, idx = self._genOption()

        # Find the column with the simulationPeriod
        idSimPer = np.where(data_gen.values[0, :] == self.resources['period'])[0][0]
        data_gen.drop(data_gen.columns[idSimPer+1:], axis=1, inplace=True)

        # Find the id number of Generators and Bus location
        idGenInfo = np.array([False for i in np.arange(idSimPer+1)])
        idGenInfo[:6] = True

        genInfo = data_gen.values[:, idGenInfo]
        for i in np.arange(genInfo.shape[0]):
            genInfo[i, :] = pd.to_numeric(genInfo[i, :], errors='coerce')

        numListGen1 = int(np.nanmax(genInfo[:, 1]))
        numListGen2 = int(np.nanmax(genInfo[:, genInfo.shape[1]-2]))
        numListGen = max(numListGen1, numListGen2)

        # Separate the information of each generator
        data_gen = pd.DataFrame(data_gen.values[:, idGenInfo==False])
        ids = pd.isnull(data_gen[data_gen.columns[0]])
        ids_values = np.where(ids == True)[0] + 1
        data_gen = data_gen.values[ids_values[0]:, :]

        ids = pd.isnull(data_gen[:, 0])
        data_gen = data_gen[ids==False, :]

        # Eliminate the NaN elements
        idNanElem = pd.isnull(genInfo[:, 1])
        idNanElemText = pd.isnull(genInfo[:, 0])
        genInfo = genInfo[idNanElem == False, :]
        genInfo = genInfo[:, [0, 1, 3]]

        # Determine the number of generators
        numGen = int(np.nanmax(genInfo[:, 0]))
        idGen = np.arange(1, numGen + 1)
        numListGen2 = int(data_gen.shape[0] / numGen)

        # Pass the excel values to the matlab structure
        generator = {}
        temp_limit = np.zeros((numGen, data_gen.shape[1], numListGen2))
        for i in np.arange(1, numGen+1):
            if (genInfo[int((i-1) * numListGen1 + 1), 2]) == 1:
                temp_limit[i-1, :, 0] = genInfo[int((i-1) * numListGen1 + 5), 2]
            else:
                temp_limit[i-1, :, 0] = data_gen[int((i-1) * numListGen2), :]

        for i in np.arange(2, numListGen2+1):
            temp_limit[:, :, i-1] = data_gen[(idGen-1) * numListGen2 + (i-1), :]
        generator['limit'] = temp_limit

        # Owner Info
        owner = {}
        temp_idx = np.array((idGen-1) * numListGen1 + 2, dtype=int)
        owner['gen'] = (np.array(genInfo[temp_idx, 2], dtype=int))

        # Generator Info
        infoGen = genInfo[:, 2]
        idx = pd.isnull(infoGen)
        infoGen[idx] = 0
        infoGen = np.array(infoGen, dtype=int)
        # Matlab uses Fortran-like indexing
        infoGen = np.reshape(infoGen, (numListGen1, numGen), order='F')
        generator['info'] = infoGen

        # Generator Location set
        genInBus = np.zeros((max(self.pD_network['busNum'].transpose())[0], numGen))
        idx = pd.isnull(genInfo[:, 0]) == False
        idx = np.where(idx == True)[0]

        idGenLoc = np.array(genInfo[idx, 2], dtype=int)
        idGenLoc = np.ravel_multi_index([idGenLoc.transpose(), idGen-1],
                                        dims=(genInBus.shape[0], genInBus.shape[1]),
                                        order='F') - 1

        temp_genInBus = genInBus.ravel()
        temp_genInBus[np.array(idGenLoc)] = 1
        temp_genInBus = np.reshape(temp_genInBus,
                                   (genInBus.shape[0], genInBus.shape[1]),
                                   order='F')
        resources = {'genInBus': temp_genInBus,
                     'numGen': numGen}

        self.generator = generator
        self.parameterData['resources'] = resources

        return


    def readExcel(self):

        # Excel sheets to read from the excel
        sheet_name_branch = 'Network_Info'
        sheet_name_peers = 'Peers_Info_EC{}'.format(self.ecID)
        sheet_name_gen = 'Generator_EC{}'.format(self.ecID)
        sheet_name_load = 'Load_EC{}'.format(self.ecID)
        sheet_name_stor = 'Storage_EC{}'.format(self.ecID)
        sheet_name_cstation = 'CStation_EC{}'.format(self.ecID)
        sheet_name_v2g = 'Vehicle_EC{}'.format(self.ecID)

        #data_peers = pd.read_excel(path, sheet_name=sheet_name_peers, header=None)
        #data_gen = pd.read_excel(path, sheet_name=sheet_name_gen, header=None)
        #data_load = pd.read_excel(path, sheet_name=sheet_name_load, header=None)
        #data_storage = pd.read_excel(path, sheet_name=sheet_name_stor, header=None)
        #data_cstation = pd.read_excel(path, sheet_name=sheet_name_cstation, header=None)
        #data_v2g = pd.read_excel(path, sheet_name=sheet_name_v2g, header=None)

        return

data = ExcelReader(path='src/EC_OnlyMembers_V8/EC_V4.xlsx', ecID=1)

In [ ]:
data.network['busRef']